# Exercises

## Exercise 1
---

You can find below the code that was used to generate the activity of place cells on a linear track.
Use the code and the decoding procedure you lerned about in the lesson to explore how different characteristic of the data impact our ability to decode position.
In particular:

A - Try to use different fraction of our data samples. How does the median error change when the the number of available sample gets larger? You do not need to re-generate any data, just randomly sub-sample the data to different fractions.

B - How many place cells do we need to reliably decode? Try to re-do the decoding using only 10 cell, then 20, and so on. How does the median error change? Does it reach an asymptote? (Also in this case, you do not need to re-generate the data, you can just select a random subset of cells each time)

C - Generate new data using the code below, changing the firing rate noise (changing the value of the variable `noise firing_rate`). How does this noise impact the decoding? 

In [1]:
import numpy as np
import scipy
import scipy.stats
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import seaborn as sns


%matplotlib inline 
plt.rcParams['figure.figsize'] = [10, 5]
from ipywidgets import interact
import ipywidgets as widgets


track_length = 200. # the length of our linear track (eg in centimeter)
average_firing_rate = 5 # the peak firing rate, averaged across the population 
n_cells = 100 # how many cells we are recording
pf_centers = np.random.rand(n_cells) * track_length # the centers of the place fields for all cells drawn randomly with a uniform distribution on the track
pf_size = np.random.gamma(10, size=n_cells) # the size (width) of the place fields, drawn randomly from a gamma distribution 
pf_rate = np.random.exponential(scale=average_firing_rate, size=n_cells) # the peak firing rate for each cell, drawn from an exponential distribution

In [2]:
bins = np.arange(0., 200.)
true_firing_rate_maps = np.zeros((n_cells, len(bins)))
for i in range(n_cells):
    true_firing_rate_maps[i,:] = pf_rate[i] * np.exp(-((bins-pf_centers[i])**2)/(2*pf_size[i]**2))

In [3]:
# GENERATE TRAJECTORY

n_runs = 10
use_stops = False
av_running_speed = 10 # the average running speed (in cm/s)
fps = 10 # the number of "video frames" per second 
running_speed_a = np.random.chisquare(10, size=n_runs) # running speed in the two directions
running_speed_b = np.random.chisquare(10, size=n_runs) 

stopping_time_a = np.random.chisquare(15, size=n_runs) # the time the mouse will spend at the two ends of the track
stopping_time_b = np.random.chisquare(15, size=n_runs)

x = np.array([])


for i in range(n_runs):
    stop1 = np.ones((int(stopping_time_a[i]*fps),)) * 0.
    run_length = len(bins) * fps / running_speed_a[i]
    run1 = np.linspace(0., float(len(bins)-1), int(run_length))
    stop2 = np.ones((int(stopping_time_b[i]*fps),)) * (len(bins)-1.)
    run_length = len(bins) * fps / running_speed_b[i]
    run2 = np.linspace(len(bins)-1., 0., int(run_length))
    if use_stops:
        x = np.concatenate((x, stop1, run1, stop2, run2))
    else:
         x = np.concatenate((x, run1, run2))
t = np.arange(len(x))/fps

In [4]:
sampling_rate = 10000.
t_sampling = np.arange(0, t[-1], 1. / sampling_rate)
x_sampling = np.floor(np.interp(t_sampling, t, x))
noise_firing_rate = 0.1 # the baseline noise firing rate =0.1
#noise_firing_rate = 1
spikes = []

for i in range(n_cells):
    inst_rate = true_firing_rate_maps[i,x_sampling.astype(np.int32)] + noise_firing_rate
    spikes_loc = np.random.poisson(inst_rate/sampling_rate)
    sp = np.argwhere(spikes_loc)
    t_sp = t_sampling[sp]
    spikes.append(t_sp)

In [5]:
file_name = 'linear_track_data.pickle' # change this name when you save new data
#file_name = 'linear_track_data.picklenoise1' 


out_data = {}
out_data['x'] = x
out_data['t'] = t
out_data['spikes'] = spikes
out_data['track_length'] = track_length
out_data['fps'] = fps

with open('data/'+file_name,'wb') as f:
    pickle.dump(out_data,f)

In [6]:
len(spikes)

100

In [7]:
#spikes is a list of 100 slots, as many as the n_cells, so 1% is 1, 5% is 5 etc. 
spikes1=spikes[90]
spikes5=spikes[37:42]
spikes10=spikes[79:89]
spikes20=spikes[80:100]
spikes50=spikes[50:]
spikes5050=spikes[:50]





# we compute the poistion at which each spike was emitted
spike_positions = [np.interp(s, t, x) for s in spikes]

spike_positions1 = [np.interp(s, t, x) for s in spikes1]
spike_positions5 = [np.interp(s, t, x) for s in spikes5]
spike_positions10 = [np.interp(s, t, x) for s in spikes10]
spike_positions20 = [np.interp(s, t, x) for s in spikes20]
spike_positions50 = [np.interp(s, t, x) for s in spikes50]
spike_positions5050 = [np.interp(s, t, x) for s in spikes5050]


space_bins = np.arange(0., track_length, 5.) # binnin in bins of 5 cms

# we compute histograms for each cell
spikes_hist= [np.histogram(s, space_bins)[0] for s in spike_positions]
spikes_hist = np.asarray(spikes_hist)

spikes_hist1= [np.histogram(s, space_bins)[0] for s in spike_positions1]
spikes_hist1 = np.asarray(spikes_hist1)
spikes_hist5= [np.histogram(s, space_bins)[0] for s in spike_positions5]
spikes_hist5 = np.asarray(spikes_hist5)
spikes_hist10= [np.histogram(s, space_bins)[0] for s in spike_positions10]
spikes_hist10 = np.asarray(spikes_hist10)
spikes_hist20= [np.histogram(s, space_bins)[0] for s in spike_positions20]
spikes_hist20 = np.asarray(spikes_hist20)
spikes_hist50= [np.histogram(s, space_bins)[0] for s in spike_positions50]
spikes_hist50 = np.asarray(spikes_hist50)
spikes_hist5050= [np.histogram(s, space_bins)[0] for s in spike_positions5050]
spikes_hist5050 = np.asarray(spikes_hist5050)

# we also need an "occupancy histogram" in order to normalize the firing rates maps 
occupancy = np.histogram(x, space_bins)[0] /  fps

firing_rate_maps = spikes_hist / occupancy 

firing_rate_maps1 = spikes_hist1 / occupancy 
firing_rate_maps5 = spikes_hist5 / occupancy
firing_rate_maps10 = spikes_hist10 / occupancy 
firing_rate_maps20 = spikes_hist20 / occupancy 
firing_rate_maps50 = spikes_hist50 / occupancy
firing_rate_maps5050 = spikes_hist5050 / occupancy

In [8]:
spikes_count= [np.histogram(s,t)[0] for s in spikes]
spikes_count = np.asarray(spikes_count).T # we transpose the matrix to have the more familiar samples x features shape

spikes_count1= [np.histogram(s,t)[0] for s in spikes1]
spikes_count1 = np.asarray(spikes_count1).T
spikes_count5= [np.histogram(s,t)[0] for s in spikes5]
spikes_count5 = np.asarray(spikes_count5).T
spikes_count10= [np.histogram(s,t)[0] for s in spikes10]
spikes_count10= np.asarray(spikes_count10).T
spikes_count20= [np.histogram(s,t)[0] for s in spikes20]
spikes_count20 = np.asarray(spikes_count20).T
spikes_count50= [np.histogram(s,t)[0] for s in spikes50]
spikes_count50= np.asarray(spikes_count50).T
spikes_count5050= [np.histogram(s,t)[0] for s in spikes5050]
spikes_count5050= np.asarray(spikes_count5050).T

In [9]:
#prior = occupancy / sum(occupancy)
true_x = x[:-1] 




decoding_times = t[:-1]
epsilon = pow(1,-10)
log_posteriors = spikes_count @ np.log(firing_rate_maps+epsilon) - (1./fps)*np.sum(firing_rate_maps, axis=0) #+ np.log(prior + epsilon)

log_posteriors1 = spikes_count1 @ np.log(firing_rate_maps1+epsilon) - (1./fps)*np.sum(firing_rate_maps1, axis=0) #+ np.log(prior + epsilon)
log_posteriors5 = spikes_count5 @ np.log(firing_rate_maps5+epsilon) - (1./fps)*np.sum(firing_rate_maps5, axis=0) #+ np.log(prior + epsilon)
log_posteriors10 = spikes_count10 @ np.log(firing_rate_maps10+epsilon) - (1./fps)*np.sum(firing_rate_maps10, axis=0) #+ np.log(prior + epsilon)
log_posteriors20 = spikes_count20 @ np.log(firing_rate_maps20+epsilon) - (1./fps)*np.sum(firing_rate_maps20, axis=0) #+ np.log(prior + epsilon)
log_posteriors50 = spikes_count50 @ np.log(firing_rate_maps50+epsilon) - (1./fps)*np.sum(firing_rate_maps50, axis=0) #+ np.log(prior + epsilon)
log_posteriors5050 = spikes_count5050 @ np.log(firing_rate_maps5050+epsilon) - (1./fps)*np.sum(firing_rate_maps5050, axis=0) #+ np.log(prior + epsilon)


x_parallel = [space_bins[np.argmax(P)] for P in log_posteriors]

x_parallel1 = [space_bins[np.argmax(P)] for P in log_posteriors1]
x_parallel5 = [space_bins[np.argmax(P)] for P in log_posteriors5]
x_parallel10 = [space_bins[np.argmax(P)] for P in log_posteriors10]
x_parallel20 = [space_bins[np.argmax(P)] for P in log_posteriors20]
x_parallel50 = [space_bins[np.argmax(P)] for P in log_posteriors50]
x_parallel5050 = [space_bins[np.argmax(P)] for P in log_posteriors5050]



# error distribution
mse = np.sqrt((true_x-x_parallel)**2)

mse1=np.sqrt((true_x-x_parallel1)**2)
mse5=np.sqrt((true_x-x_parallel5)**2)
mse10=np.sqrt((true_x-x_parallel10)**2)
mse20=np.sqrt((true_x-x_parallel20)**2)
mse50=np.sqrt((true_x-x_parallel50)**2)
mse5050=np.sqrt((true_x-x_parallel5050)**2)


#sns.histplot(mse10)
#plt.axvline(x = np.nanmedian(mse),c='r')
print(f'Median error: {np.nanmedian(mse)} cm')
print(f'Median error1%: {np.nanmedian(mse1)} cm')
print(f'Median error5%: {np.nanmedian(mse5)} cm')
print(f'Median error10%: {np.nanmedian(mse10)} cm')
print(f'Median error20%: {np.nanmedian(mse20)} cm')
print(f'Median error last 50%: {np.nanmedian(mse50)} cm')
print(f'Median error first 50%: {np.nanmedian(mse5050)} cm')

Median error: 4.402234636871498 cm
Median error1%: 98.66386554621849 cm
Median error5%: 23.70891089108909 cm
Median error10%: 26.74083129584352 cm
Median error20%: 18.022255192878333 cm
Median error last 50%: 7.009779951100242 cm
Median error first 50%: 5.992665036674815 cm


In [29]:
# After running 2 simulations with different noise levels I got these results:

# Noise = 0.1

#Median error: 3.88032258064516 cm
#Median error1%: 53.33609809487978 cm
#Median error5%: 41.604781172584644 cm
#Median error10%: 19.184588849524516 cm
#Median error20%: 13.366241159220294 cm
#Median error last 50%: 5.696522024983572 cm
#Median error first 50%: 5.986930837647691 cm

#Noise = 1

#Median error: 4.220312623352015 cm
#Median error1%: 65.09205020920501 cm
#Median error5%: 31.394949170679514 cm
#Median error10%: 24.19741252522489 cm
#Median error20%: 18.62509471542075 cm
#Median error last 50%: 6.64414258188825 cm
#Median error first 50%: 8.561682723185612 cm

<font color='blue'>Our results show that with a bigger sample, we get smaller errors. However, it's been noticed that the error depends on the "area" of the sample,first 50% has a higher error than last 50% of the sample. If we increase the noise (by 10 times, from 0,1 to 1) we get increased ms errors in our decoding. </font>

In [39]:
# How many cells to reliably decode ?

# We saw that at 20% the error is still double digits and even at 50% it's still a third bigger
# than our original error (mse~4). So we don't expect much from 10, 20 cells.
# ! Reminder ! len(spikes)=100 so 50% is 50 cells etc. 
# I take increments of y cells and calculate the mse of y cells along the whole sample.
mses=[]
y=45

for i in range(100):
    spikeshm=spikes[i:i+y]
    if i+y<101:
        

        spike_positionshm = [np.interp(s, t, x) for s in spikeshm]

        spikes_histhm= [np.histogram(s, space_bins)[0] for s in spike_positionshm]
        spikes_histhm = np.asarray(spikes_histhm)

        firing_rate_mapshm = spikes_histhm / occupancy 

        spikes_counthm= [np.histogram(s,t)[0] for s in spikeshm]
        spikes_counthm = np.asarray(spikes_counthm).T

        log_posteriorshm = spikes_counthm @ np.log(firing_rate_mapshm+epsilon) - (1./fps)*np.sum(firing_rate_mapshm, axis=0) #+ np.log(prior + epsilon)
        x_parallelhm = [space_bins[np.argmax(P)] for P in log_posteriorshm]
        msehm=np.sqrt((true_x-x_parallelhm)**2)

        mses.append(np.nanmedian(msehm))

        print(f'Median error for[{i}]: {np.nanmedian(msehm)} cm')
    else:
        print('Exceeds limit')
print(f'Median error for total: {np.nanmedian(mses)} cm')
print(f'Standard deviation of total: {np.std(mses)}')


Median error for[0]: 6.601476014760152 cm
Median error for[1]: 6.7049180327868925 cm
Median error for[2]: 7.167597765363126 cm
Median error for[3]: 6.723628691983123 cm
Median error for[4]: 6.675105485232066 cm
Median error for[5]: 6.607920792079213 cm
Median error for[6]: 6.541176470588255 cm
Median error for[7]: 6.467326732673257 cm
Median error for[8]: 6.542787286063572 cm
Median error for[9]: 6.593147751605997 cm
Median error for[10]: 6.7995780590717345 cm
Median error for[11]: 6.7724288840262545 cm
Median error for[12]: 6.783382789317528 cm
Median error for[13]: 6.5935397039031045 cm
Median error for[14]: 6.59299781181619 cm
Median error for[15]: 6.606060606060609 cm
Median error for[16]: 6.390099009900979 cm
Median error for[17]: 6.202933985330077 cm
Median error for[18]: 6.257065948855981 cm
Median error for[19]: 6.7671232876712395 cm
Median error for[20]: 7.042826552462529 cm
Median error for[21]: 6.970588235294116 cm
Median error for[22]: 6.7927321668909855 cm
Median error for

<font color='blue'> By executing the cell above numerous times I started noticing a decreasing mse pattern even from 10 cells.However the mse and its fluctuations were still high. After 32 cells (y=32) the mse was mostly under 10 and around 45 cells the fluctuations became smaller (standard deviation). I would have to say we need at least half of the sample to have a similar decoding result as the whole sample. </font>